## 1. load package

In [ ]:
import MLJ:predict,predict_mode,measures
using   MLJ, CSV, DataFrames,Tidier
using  CairoMakie,AlgebraOfGraphics,MakieThemes
using CatBoost.MLJCatBoostInterface

## 2. load csv

In [ ]:
df=CSV.File("../../data/clustering-datasets/basic2.csv")|>DataFrame
df=@chain  df  begin
    @clean_names
    #coerce(_,:color=> Multiclass)
end
first(df,5)

## 3. describe df

In [ ]:
describe(df)

## 4. tidy data

In [ ]:
cat=levels(df.color)
ytrain, Xtrain =  unpack(df, ==(:color), rng=123);
size(ytrain)

## 5. import test data 

In [ ]:
include("../utils.jl")  # import boundary_data
tx,ty,xtest=boundary_data(df)

## 6. MLJ CatBoost classification workflow
### 6.1 load model 

In [ ]:
catboost = CatBoostClassifier(iterations=2,learning_rate=0.20)
mach = machine(catboost, Xtrain, ytrain)|>fit!

### 6.2 predict test data 

In [ ]:
ytest = predict_mode(mach, xtest)[:,1]|>Array
show(ytest)

### 6.3 plot results

In [ ]:
fig=Figure(resolution=(600,600))
ax=Axis(fig[1,1])
contourf!(ax, tx,ty,ytest)
train_data= data(df)
maplayer=mapping(:x,:y, color=:color)
vislayer=visual(Scatter,strokewidth=1,strokecolor=:black)
plt=train_data*maplayer*vislayer
draw!(fig[1,1],plt)
fig